## Image Augmentation at runtime
Using ImageDataGenerator from [keras image preprocessing](https://keras.io/preprocessing/image/#image-preprocessing). Watch out overfeatting on lerning history.

In [ ]:
datagen = ImageDataGenerator(
    # instead of  rescale=1./255
    featurewise_center=True,
    featurewise_std_normalization=True,
    # randomly select transformation within given limits in %
    rotation_range=20,     # 20%
    width_shift_range=0.2, # 20%
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nerest'
)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


Overfit example and augmentation n action [Cats vs Dogs](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb)

Note, you need image **variations in training input data** and similary variations needs to be in your **test/validation set as well**, otherwise validation score would flactuate dramatically and not represent actual level of confidence.

# Transfer learning

Example in colab [Transfer Learning.ipynb](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb) same as used in tutorial [Transfer Learning on tensorflow.org](https://www.tensorflow.org/tutorials/images/transfer_learning).

create the base model from [prebuilt repo in keras](https://keras.io/applications/#applications): 
Xception
VGG16
VGG19
ResNet, ResNetV2
InceptionV3
InceptionResNetV2
MobileNet
MobileNetV2
DenseNet
NASNet
.

**Transfer base model** for feature extraction 
```python
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet' # use built in weights
                                              )
# Freeze the Feature extraction convolutional base before we compile and train the model. 
base_model.trainable = False

# Add a classification head
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.compile()
history = model.fit_generator(train_generator, ..)
```

* Weights can be saved/loaded localy as snapshot
```python
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None
                                              )
base_model.load_weights('/tmp/kernels_v123.h5')
```


* Some models may require special pre-processing and post-processing
```python
from keras.applications.resnet50 import preprocess_input, decode_predictions
```

**Fine tuning layers** You can start fine tune few top layers after base model trained to converge on your data.
```python
print("Number of layers in the base model: ", len(base_model.layers))
# Fine tune from this layer onwards
fine_tune_at = 100
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
# alternative reference by name
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
```

* Note, the room for fine tuning may be limited by overfitting
![fine-tune-loss.png](fine-tune-loss.png)

* Don't forget add Dropout layers into your classifier to fight overfitting. [Dropout regularization note by Ng](https://www.youtube.com/watch?v=ARq74QuavAo). Seting dropout level too high may result in NN would lose specialization to the effect that it would be inefficient or ineffective at learning, driving accuracy down.

**NASNet** paper [2017 Learning Transferable Architectures for Scalable Image Recognition](https://arxiv.org/abs/1707.07012)
